In [ ]:
# EPenv\Scripts\activate

# exoplanetarchive

Retrieval

In [ ]:
import requests
import pandas as pd

#all fields and rows from ps database
query_url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+*+from+ps&format=csv"

response = requests.get(query_url)

if response.status_code == 200:
    csv_filename = "exoplanetarchive.csv"
    
    with open(csv_filename, "wb") as f:
        f.write(response.content)
    
    df = pd.read_csv(csv_filename)
    
else:
    print(f"Error fetching data: {response.status_code}")


print(df.head())

C:\Users\farah\AppData\Local\Temp\ipykernel_19528\2782889823.py:19: DtypeWarning: Columns (3,4,50,65,153,183,187,206,211) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filename)


First 5 rows of the dataset:
      pl_name pl_letter  hostname hd_name hip_name        tic_id  \
0  Kepler-6 b         b  Kepler-6     NaN      NaN  TIC 27916356   
1  Kepler-6 b         b  Kepler-6     NaN      NaN  TIC 27916356   
2  Kepler-6 b         b  Kepler-6     NaN      NaN  TIC 27916356   
3  Kepler-6 b         b  Kepler-6     NaN      NaN  TIC 27916356   
4  Kepler-6 b         b  Kepler-6     NaN      NaN  TIC 27916356   

                        gaia_id  default_flag  \
0  Gaia DR2 2086636884980514304             0   
1  Gaia DR2 2086636884980514304             0   
2  Gaia DR2 2086636884980514304             1   
3  Gaia DR2 2086636884980514304             0   
4  Gaia DR2 2086636884980514304             0   

                                          pl_refname  \
0  <a refstr=MORTON_ET_AL__2016 href=https://ui.a...   
1  <a refstr=EXOFOP_TESS_TOI href=https://exofop....   
2  <a refstr=ESTEVES_ET_AL__2015 href=https://ui....   
3  <a refstr=Q1_Q12_KOI_TABLE href=https://

Cleaning

In [ ]:
import pandas as pd
import numpy as np

csv_filename = "exoplanetarchive.csv"
df = pd.read_csv(csv_filename)

#rows where default = 1
df = df[df["default_flag"] == 1]

#remove columns with "err1", "err2", "lim", "str", or "ref"
columns_to_remove = [col for col in df.columns if any(sub in col for sub in ["err1", "err2", "lim", "str", "ref"])]
df = df.drop(columns=columns_to_remove)

#spectral type simplification
def get_first_letter(val):
    if pd.isna(val):
        return np.nan
    if isinstance(val, str) and val.strip():
        return val.strip()[0]
    return np.nan

df['st_spectype_simp'] = df['st_spectype'].apply(get_first_letter)

cleaned_csv_filename = "clean_exoplanetarchive.csv"
df.to_csv(cleaned_csv_filename, index=False)


C:\Users\farah\AppData\Local\Temp\ipykernel_19528\2774513706.py:6: DtypeWarning: Columns (3,4,50,65,153,183,187,206,211) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filename)


First 5 rows of the cleaned dataset:
         pl_name pl_letter    hostname hd_name hip_name         tic_id  \
2     Kepler-6 b         b    Kepler-6     NaN      NaN   TIC 27916356   
21  Kepler-491 b         b  Kepler-491     NaN      NaN  TIC 158388163   
34  Kepler-257 b         b  Kepler-257     NaN      NaN  TIC 273132699   
42  Kepler-216 b         b  Kepler-216     NaN      NaN  TIC 270698499   
59   Kepler-32 c         c   Kepler-32     NaN      NaN  TIC 273590427   

                         gaia_id  default_flag disc_pubdate  disc_year  ...  \
2   Gaia DR2 2086636884980514304             1      2010-04       2009  ...   
21  Gaia DR2 2102468134431912064             1      2016-05       2016  ...   
34  Gaia DR2 2080246282881778048             1      2014-03       2014  ...   
42  Gaia DR2 2128067720065692672             1      2014-03       2014  ...   
59  Gaia DR2 2080287892525359872             1      2012-05       2011  ...   

      sy_pmra   sy_pmdec    sy_plx   sy_dis

Calculations and Decisions

In [ ]:
# 1 if rocky (using planet radius in Earth units)

import pandas as pd
import numpy as np

df = pd.read_csv('clean_exoplanetarchive.csv')

if 'pl_rade' in df.columns:
    df['is_rocky'] = np.where(df['pl_rade'] < 1.6, 1, 0)

df.to_csv('p_clean_exoplanetarchive.csv', index=False)

In [ ]:
#ESI

import pandas as pd
import numpy as np

df = pd.read_csv('p_clean_exoplanetarchive.csv')

#constants
S_EARTH = 1.0
R_EARTH = 1.0

def compute_esi(S, R):

    if pd.isna(S) or pd.isna(R) or S <= 0 or R <= 0:
        return np.nan
 
    val = 1 - np.sqrt((1/2)*(((S-S_EARTH)/(S+S_EARTH))**2 + ((R-R_EARTH)/(R+R_EARTH))**2))
    return val

def get_radius(row):

    r = row.get('pl_rade', np.nan)
    if pd.notna(r):
        return r
    
    #if there's no radius
    m = row.get('pl_bmasse', np.nan)
    if pd.notna(m) and m > 0:
        return m ** (1.0 / 3.0)
    
    return np.nan


df['pl_radius_for_esi'] = df.apply(get_radius, axis=1)

#calculate ESI
df['pl_esi'] = df.apply(
    lambda row: compute_esi(
        S=row.get('pl_insol', np.nan),
        R=row['pl_radius_for_esi']
    ),
    axis=1
)

df.to_csv('p_clean_exoplanetarchive.csv', index=False)


In [ ]:
#HZA

import pandas as pd
import numpy as np

df = pd.read_csv('p_clean_exoplanetarchive.csv')

#constants
Z = 2e-2
M_WN = 14
M_WH = 1

def compute_hza(m, r, Teq):

    if np.isnan(m) or np.isnan(r) or np.isnan(Teq) or r <= 0 or Teq <= 0:
        return np.nan
    
    v_eN = np.sqrt(Z * Teq / M_WN)
    v_eH = np.sqrt(Z * Teq / M_WH)

    numerator = 2 * np.sqrt(m / r) - v_eH - v_eN
    denominator = v_eH - v_eN
    
    if denominator == 0:
        return np.nan
    
    return numerator / denominator

#calculate HZA
df['pl_hza'] = df.apply(
    lambda row: compute_hza(
        row.get('pl_bmasse', np.nan),
        row.get('pl_rade', np.nan),
        row.get('pl_eqt', np.nan)
    ),
    axis=1
)

df.to_csv('p_clean_exoplanetarchive.csv', index=False)

In [ ]:
# HZC

import pandas as pd
import numpy as np

df = pd.read_csv('p_clean_exoplanetarchive.csv')

#constants
IRON = {
    'm1': 5.80,
    'r1': 2.52,
    'k1': -0.209490,
    'k2': 0.0804,
    'k3': 0.394
}

#
WATER = {
    'm1': 5.52,
    'r1': 4.43,
    'k1': -0.209396,
    'k2': 0.0807,
    'k3': 0.375
}

def radius_composition(m, composition):

    if m <= 0:
        return np.nan
    
    c = composition
    m_ratio = m / c['m1']
    
    exponent = c['k1'] + (1/3.0)*np.log10(m_ratio) - c['k2']*(m_ratio**c['k3'])
    return c['r1'] * (10 ** exponent)

def compute_hzc(m, r):

    if np.isnan(m) or np.isnan(r):
        return np.nan
    #make sure radius (in earth units) doesn't exceed 20
    if m <= 0 or r <= 0 or m > 20:
        return np.nan
    
    r_i = radius_composition(m, IRON)
    r_o = radius_composition(m, WATER)
    
    if np.isnan(r_i) or np.isnan(r_o):
        return np.nan
    
    denominator = r_o - r_i
    if denominator == 0:
        return np.nan
    
    numerator = 2*r - r_o - r_i
    return numerator / denominator

#calculate HZC
df['pl_hzc'] = df.apply(
    lambda row: compute_hzc(
        row.get('pl_bmasse', np.nan),
        row.get('pl_rade', np.nan)
    ),
    axis=1
)

df.to_csv('p_clean_exoplanetarchive.csv', index=False)

In [ ]:
# HZD

import pandas as pd
import numpy as np

df = pd.read_csv('p_clean_exoplanetarchive.csv')

#constants
T_s = 5700.0

a_i = 2.7619e-5
b_i = 3.8095e-9
r_is = 0.72

a_o = 1.3786e-4
b_o = 1.4286e-9
r_os = 1.77

def compute_hzd(L, T_eff, r):

    if any(pd.isna(x) for x in [L, T_eff, r]):
        return np.nan
    if L <= 0 or r <= 0 or T_eff <= 0:
        return np.nan
    
    delta_T = T_eff - T_s
    
    r_i = (r_is - a_i*delta_T - b_i*(delta_T**2)) * np.sqrt(L)
    r_o = (r_os - a_o*delta_T - b_o*(delta_T**2)) * np.sqrt(L)

    denom = r_o - r_i
    if denom == 0:
        return np.nan
    
    numerator = 2.0*r - r_o - r_i
    return numerator / denom

#calculate HZD
df['pl_hzd'] = df.apply(
    lambda row: compute_hzd(
        L=row.get('st_lum', np.nan),
        T_eff=row.get('st_teff', np.nan),
        r=row.get('pl_orbsmax', np.nan)
    ),
    axis=1
)

df.to_csv('p_clean_exoplanetarchive.csv', index=False)

Labeling

In [ ]:
import pandas as pd

archive_file = "clean_exoplanetarchive.csv"
terrestrial_file = "Types/Terrestrial.csv"
gasgiant_file = "Types/Gasgiant.csv"
neptunelike_file = "Types/Neptune-like.csv"
superearth_file = "Types/Superearth.csv"

#output
known_output_file = "known_exoplanets.csv"
unknown_output_file = "unknown_exoplanets.csv"


df_archive = pd.read_csv(archive_file)
df_terrestrial = pd.read_csv(terrestrial_file)
df_gasgiant = pd.read_csv(gasgiant_file)
df_neptunelike = pd.read_csv(neptunelike_file)
df_superearth = pd.read_csv(superearth_file)

terrestrial_set = set(df_terrestrial["Planet Name"].str.strip())
gasgiant_set = set(df_gasgiant["Planet Name"].str.strip())
neptunelike_set = set(df_neptunelike["Planet Name"].str.strip())
superearth_set = set(df_superearth["Planet Name"].str.strip())

def get_planet_type(planet_name):
    name = str(planet_name).strip()
    if name in terrestrial_set:
        return "Terrestrial"
    elif name in gasgiant_set:
        return "Gas Giant"
    elif name in neptunelike_set:
        return "Neptune-Like"
    elif name in superearth_set:
        return "Super Earth"
    else:
        return "Unknown"

df_archive["pl_type"] = df_archive["pl_name"].apply(get_planet_type)

#known
df_known = df_archive[df_archive["pl_type"] != "Unknown"]
#unknown
df_unknown = df_archive[df_archive["pl_type"] == "Unknown"]

df_known.to_csv(known_output_file, index=False)
df_unknown.to_csv(unknown_output_file, index=False)

print(f"Known exoplanets saved to '{known_output_file}'.")
print(f"Unknown exoplanets saved to '{unknown_output_file}'.")

Known exoplanets saved to 'known_exoplanets.csv'.
Unknown exoplanets saved to 'unknown_exoplanets.csv'.
